In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
import matplotlib.pyplot as plt
import torchtext

import tiktoken
import pandas as pd
import numpy as np

from torch.nn import functional as F
import torch.nn as nn
from tqdm.notebook import tqdm
import pickle
import ast

In [2]:
import os

#dataset https://nlp.stanford.edu/projects/nmt/

#tiktoken api https://github.com/openai/tiktoken
cl100k_base = tiktoken.get_encoding("cl100k_base")

# In production, load the arguments directly instead of accessing private attributes
# See openai_public.py for examples of arguments for specific encodings
tokenizer = tiktoken.Encoding(
    # If you're changing the set of special tokens, make sure to use a different name
    # It should be clear from the name what behaviour to expect.
    name="cl100k_im",
    pat_str=cl100k_base._pat_str,
    mergeable_ranks=cl100k_base._mergeable_ranks,
    special_tokens={
        **cl100k_base._special_tokens,
        "<|PAD|>": 100277,
        "<|START|>": 100278,
        "<|END|>": 100279,
        "<|DEL|>": 100280,

    }
)
print(tokenizer.n_vocab) #this is the number of tokens in our tokenizer
print(tokenizer._special_tokens) #prints out our special tokens 

specials = {"<|PAD|>","<|START|>","<|END|>", "<|DEL|>"}

print(dir(tokenizer))

100281
{'<|endoftext|>': 100257, '<|fim_prefix|>': 100258, '<|fim_middle|>': 100259, '<|fim_suffix|>': 100260, '<|endofprompt|>': 100276, '<|PAD|>': 100277, '<|START|>': 100278, '<|END|>': 100279, '<|DEL|>': 100280}
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_core_bpe', '_encode_bytes', '_encode_only_native_bpe', '_encode_single_piece', '_mergeable_ranks', '_pat_str', '_special_tokens', 'decode', 'decode_batch', 'decode_bytes', 'decode_bytes_batch', 'decode_single_token_bytes', 'decode_tokens_bytes', 'encode', 'encode_batch', 'encode_ordinary', 'encode_ordinary_batch', 'encode_single_token', 'encode_with_unstable', 'eot_token', 'max_token_value', 'n_vocab', 'name', 'special_tokens_set', '

In [22]:
#GLOBALS

block_size = 64 #This is the value of T
batch_size = 16 #This it the value of B
n_embed = 128
dropout = 0.2
n_heads = 8
n_layers = 6

learning_rate = 1e-4

eval_interval = 500
eval_iters = 200
max_iters = 50000

vocab_size = tokenizer.n_vocab
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device is: {device}")

device is: cuda


In [4]:
def create_dataset():
    num_examples = 750000

    en_max = 0
    en_length = []
    with open(os.getcwd()+'\\data\\train_en.txt', 'r', encoding='utf8') as f:
        sentences_en = []
        for i in tqdm(range(num_examples)):
            line = f.readline()
            line = line.replace("\n", "")
            len_pad = 0
            sentence = "<|PAD|> " + (line) + " <|PAD|>"
            tok_sentence = tokenizer.encode(sentence, allowed_special = specials)
            en_length.append(len(tok_sentence))
            if len(tok_sentence) > en_max:
                en_max = len(tok_sentence)
                print(en_max)

            if len(tok_sentence) <= block_size:
                len_pad = block_size - len(tok_sentence)
                tok_sentence = tok_sentence + len_pad*[100277]
                assert len(tok_sentence) == block_size, print(len(tok_sentence))
                sentences_en.append(tok_sentence)
            else:
                sentences_en.append(block_size*[100280])

    en_length = torch.tensor(en_length).float()     
    print(en_max)    
    print(f"Length of sentences: {len(sentences_en)}")

    de_max = 0
    de_length = []
    with open(os.getcwd()+'\\data\\train_de.txt', 'r', encoding='utf8') as f:
        sentences_de = []
        for i in tqdm(range(num_examples)):
            line = f.readline()
            line = line.replace("\n", "")
            len_pad = 0
            sentence = "<|START|> " + (line) + " <|END|>"
            tok_sentence = tokenizer.encode(sentence, allowed_special = specials)
            de_length.append(len(tok_sentence))
            if len(tok_sentence) > de_max:
                de_max = len(tok_sentence)
                print(de_max)

            if len(tok_sentence) <= block_size:
                len_pad = block_size - len(tok_sentence)
                tok_sentence = tok_sentence + len_pad*[100277]
                assert len(tok_sentence) == block_size, print(len(tok_sentence))
                sentences_de.append(tok_sentence)
            else:
                sentences_de.append(block_size*[100280])

    de_length = torch.tensor(de_length).float()               
    print(de_max) 
    print(f"Length of sentences: {len(sentences_de)}")

    print("Removing sentences whos length is greater than our block_size")

    #combine the arrays together
    sentences = np.array([sentences_en, sentences_de])
    #check for indices in both sentences that have rows containing the DEL token
    idx = np.where(sentences == 100280)

    #delete every row that contains the DEL token
    sentences = np.delete(sentences, idx[1], axis = 1)

    #splitting to german and english

    sentences_en = torch.tensor(sentences[0], dtype=torch.long)
    sentences_de = torch.tensor(sentences[1], dtype=torch.long)

    print(f"Length of new english sentences: {len(sentences_en)}")
    print(f"Length of new german sentences: {len(sentences_de)}")

    print(f"Average length of english tokenized sentence: {torch.mean(en_length):.4f}, and with std: {torch.std(en_length):.4f}")
    print(f"Average length of german tokenized sentence: {torch.mean(de_length):.4f}, and with std: {torch.std(de_length):.4f}")

    with open(os.getcwd()+'\\data\\english_sentences.pkl', 'wb') as f:
        pickle.dump(sentences_en, f)

    with open(os.getcwd()+'\\data\\german_sentences.pkl', 'wb') as f:
        pickle.dump(sentences_de, f)


In [5]:
create = False
if create:
    create_dataset()

In [6]:
#TRAIN AND VAL DATASETS

with open(os.getcwd()+'\\data\\english_sentences.pkl', 'rb') as f:
    english_sentences = pickle.load(f)

with open(os.getcwd()+'\\data\\german_sentences.pkl', 'rb') as f:
    german_sentences = pickle.load(f)

In [7]:
#BATCH LOADER
n = int(0.9*len(english_sentences))

train_data_en = english_sentences[:n]
val_data_en = english_sentences[n:]

train_data_de = german_sentences[:n]
val_data_de = german_sentences[n:]

def get_batch(split):
    xdata = train_data_en if "train" else train_data_en
    ydata = train_data_de if "train" else val_data_de
    idx = torch.randint(len(xdata), (batch_size,))
    #print(idx)
    x = torch.stack([xdata[i] for i in idx])
    y = torch.stack([ydata[i] for i in idx])

    #shifting our targets by 1 to the right
    t = y[:, 1:]
    #to pad the last dimension of the input tensor, pad has the form (padding_left, padding_right)
    t = F.pad(input = t, pad = (0,1,0,0), mode = 'constant', value = 0)

    x, y, t = x.to(device), y.to(device), t.to(device)

    return x, y, t


In [8]:
import math
def PositionalEncoding(seq_len, n_embd):
        
    pos_enc = torch.zeros(seq_len, n_embd)
    position = torch.arange(0, seq_len, dtype = torch.float32).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, n_embd, 2) * (-math.log(10000.0) / n_embd))
    pos_enc[:, 0::2] = torch.sin(position * div_term)
    pos_enc[:, 1::2] = torch.cos(position * div_term)

    return pos_enc.to(device)

def get_padding_mask_matrix(x, x_embed):
    """
    x is the (B, T) tokenized matrix with padding included
    x_embed is the embedded matrix that we will convert all rows to zero based on the corresponding row index = padding index
    """

    #locate every index in each tokenized sentence which contains the pad index
    pad_indices = torch.nonzero(x == 100277).squeeze().to(device) #This will return a (N, 2) where the first column represents the sentence (batch_index) and the second column represents the corresponding index which is the pad index (This 2nd column represents which row we will set to all zeros)

    #initialize a torch.ones of the shape of the embedding matrix
    mask = torch.ones(x_embed.shape).to(device)

    #For each row in the pad_indices matrix, we go to pad_indices[0] to grab the current batch example, and we go to the corresponding row of the batch example using the value of pad_indices[1]. We turn every column of that row into zeros
    #ex: if the current row is [1, 4], then we go to the 2nd batch example, go to the 4th row, and wipe it clean with zeroes
    mask[pad_indices[:,0], pad_indices[:,1], :] = 0

    #element-wise product
    x_padded = x_embed * mask

    return x_padded.to(device)


In [9]:
class Head(nn.Module):
    def __init__(self, head_size, decoder = False):
        super().__init__()
        self.decoder = decoder
        self.head_size = head_size
        self.Wk = nn.Linear(n_embed, head_size, bias = False)
        self.Wq = nn.Linear(n_embed, head_size, bias = False)
        self.Wv = nn.Linear(n_embed, head_size, bias = False)

        if self.decoder:
            self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        #assume input is of size (B, T, C)
        K = self.Wk(x) #(B, T, head_size)
        Q = self.Wq(x) #(B, T, head_size)
        V = self.Wv(x) #(B, T, head_size)

        attention_scores = Q @ K.transpose(-2, -1) * 1/(self.head_size)**(1/2) #(B, T, T)

        if self.decoder:
            attention_scores = attention_scores.masked_fill(self.tril[:T, :T] == 0, float('-inf')) #(B, T, T)
        attention_scores = F.softmax(attention_scores, dim = -1) #(B, T, T)
        scores = self.dropout(attention_scores) #(B, T, T)
        out = scores @ V #(B, T, T) @ (B, T, head_size) = (B, T, head_size)

        return out

class crossHead(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.head_size = head_size
        self.Wk = nn.Linear(n_embed, head_size, bias = False)
        self.Wq = nn.Linear(n_embed, head_size, bias = False)
        self.Wv = nn.Linear(n_embed, head_size, bias = False)

        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, enc_out):
        B, T, C = x.shape
        #assume x is of shape (B, T, C)
        #assume enc_out is of shape (B, T, C)

        K = self.Wk(enc_out) #(B, T, head_size)
        Q = self.Wq(x) #(B, T, head_size)
        V = self.Wv(enc_out) #(B, T, head_size)

        attention_scores = Q @ K.transpose(-2, -1) * 1/(self.head_size)**(1/2) #(B, T, T)
        # print("attn",attention_scores.shape)
        attention_scores = attention_scores.masked_fill(self.tril[:T, :T] == 0, float('-inf')) #(B, T, T)
        attention_scores = F.softmax(attention_scores, dim = -1) #(B, T, T)
        scores = self.dropout(attention_scores) #(B, T, T)
        out = scores @ V #(B, T, head_size)

        return out


class MultiHeadSelfAttention(nn.Module):
    def __init__(self, head_size, decoder):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, decoder) for _ in range(n_heads)])
        #output of heads is of size (B, T, n_heads*head_size)
        self.proj = nn.Linear(head_size * n_heads, n_embed)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim = -1)
        out = self.proj(out)
        out = self.dropout(out)

        return out

class MultiHeadCrossAttention(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.heads = nn.ModuleList([crossHead(head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(n_heads*head_size, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out):
        x = torch.cat([h(x, enc_out) for h in self.heads], dim = -1)
        x = self.proj(x)
        x = self.dropout(x)

        return x

class Embedding(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, n_embed, padding_idx = 100277, device = device)

    def forward(self, x):
        #assume x is of shape (B, T)
        return self.embedding(x.long()) * n_embed**(1/2)

class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(n_embed, 4*n_embed)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(4*n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.dropout(x)

        return x

class EncoderBlock(nn.Module):
    def __init__(self):
        super().__init__()
        head_size = n_embed // n_heads

        self.sa = MultiHeadSelfAttention(head_size, decoder = False)
        self.ffw = FeedForward()
        self.layernorm1 = nn.LayerNorm(n_embed)
        self.layernorm2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        #assume input x is of size (B, T, C)
        x = self.layernorm1(x) #(B, T, C)
        x = x + self.sa(x) #(B, T, C)
        x = self.layernorm2(x) #(B, T, C)
        x = x = self.ffw(x) #(B, T, C)

        return x

class DecoderCrossBlock(nn.Module):
    #one implementation of the multi head cross attention block in the decoder
    def __init__(self):
        super().__init__()
        head_size = n_embed // n_heads
        self.heads = MultiHeadCrossAttention(head_size)
        self.layernorm1 = nn.LayerNorm(n_embed)
        self.layernorm2 = nn.LayerNorm(n_embed)
        self.ffw = FeedForward()

    def forward(self, parameters):
        #assume parameters[0] is input of shape (B, T, C), It is the output of the decoder self attention layer
        #assume parameters is a list of length 2: first element is the output of the previous hidden layer, and the 2nd element is the output of the encoder
        
        x = parameters[0]
        enc_out = parameters[1]
        
        x = self.layernorm1(x) #(B, T, C)
        x = x + self.heads(x, enc_out) #(B, T, C)
        x = self.layernorm2(x) #(B, T, C)
        x = x + self.ffw(x) #(B, T, C)

        return [x, enc_out]

class DecoderSelfBlock(nn.Module):
    #one implementation of the multi head self attention block in the decoder
    def __init__(self):
        super().__init__()
        head_size = n_embed//n_heads
        self.sa = MultiHeadSelfAttention(head_size, decoder = True)
        self.layernorm1 = nn.LayerNorm(n_embed)
        self.layernorm2 = nn.LayerNorm(n_embed)
        self.ffw = FeedForward()

    def forward(self, x):
        #assume x is of shape (B, T, C)
        x = self.layernorm1(x)
        x = x + self.sa(x)
        x = self.layernorm2(x)
        x = x + self.ffw(x)

        return x

class Transformer(nn.Module):

    def __init__(self):
        super().__init__()
        self.tok_embedding_matrix_x = Embedding()
        self.tok_embedding_matrix_y = Embedding()

        #positional embedding is a function that requires no backpropagation, so we don't need to initialize it in here
        
        self.EncoderBlocks = nn.Sequential(*[EncoderBlock() for _ in range(n_layers)])
        self.DecoderSelfBlocks = nn.Sequential(*[DecoderSelfBlock() for _ in range(n_layers)])
        self.DecoderCrossBlocks = nn.Sequential(*[DecoderCrossBlock() for _ in range(n_layers)])

        self.final_layernorm = nn.LayerNorm(n_embed)
        self.final_linear = nn.Linear(n_embed, vocab_size)
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)


    #looks to be very broken
    def translate(self, x):
        #assume input is a english sentence
        tokenized_sentence = tokenizer.encode(x, allowed_special = specials)
        #make sure the sentence is less than out block size
        assert len(tokenized_sentence) <= block_size, print("this sentence is greater than our block_size")

        len_pad = block_size - len(tokenized_sentence)
        tokenized_sentence = torch.tensor(tokenized_sentence + len_pad*[0]).view(1, -1).to(device)
        run = True
        input = [100278]
        while run is True:
            input = torch.tensor(input).view(1, -1).to(device)
            T = input.shape[1]
            logits, loss = self(tokenized_sentence[:,:T], input)
            logits = logits[:, -1, :] #becomes (B, C)
            #apply softmax to get the probabilities
            probs = F.softmax(logits, dim =-1) # (B, C)
            #sample from the distribution
            idx_next = torch.multinomial(probs, num_samples = 1) # (B, 1)
            input = torch.cat((input, idx_next), dim = 1) #(B, T+1)

            if idx_next == 100279:
                run = False

        return input[0].tolist()
            
    def forward(self, x, y, targets = None):
        Bx, Tx = x.shape
        Cx = n_embed
        
        tok_embed_x = self.tok_embedding_matrix_x(x)
        pos_embed_x = PositionalEncoding(Tx, Cx)

        By, Ty, = y.shape
        Cy = n_embed

        tok_embed_y = self.tok_embedding_matrix_y(y)
        pos_embed_y = PositionalEncoding(Ty, Cy)

        tok_pos_embed_x = tok_embed_x + pos_embed_x
        tok_pos_embed_y = tok_embed_y + pos_embed_y
        
        masked_tok_embed_x = get_padding_mask_matrix(x, tok_pos_embed_x)
        masked_tok_embed_y = get_padding_mask_matrix(y, tok_pos_embed_y)

        x = masked_tok_embed_x
        y = masked_tok_embed_y

        #encoder
        enc_out = self.EncoderBlocks(x)

        #decoder self
        y = self.DecoderSelfBlocks(y)
        #decoder cross
        #its ideal to send in one parameter only (i.e self, x) when passing parameters through stacked layers in an nn.Sequential, so we have to combine our previous hidden state output along with the enc_out into one object
        y = self.DecoderCrossBlocks([y, enc_out])

        #grab the transformed decoder input from the cross attention layer
        y = y[0]
        #remaining layers
        y = self.final_layernorm(y)
        logits = self.final_linear(y)

        if targets is not None:
            logits = logits.view(By*Ty, -1)
            targets = targets.view(targets.shape[0]*targets.shape[1])
            loss = F.cross_entropy(logits, targets)

        else:
            loss = None

        return logits, loss



In [23]:
model = Transformer()
m = model.to(device)

print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

42.170425 M parameters


In [24]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1)

loss_fn = nn.CrossEntropyLoss(ignore_index = 100277)

for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

state 	 {}
param_groups 	 [{'lr': 0.0001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.01, 'amsgrad': False, 'foreach': None, 'maximize': False, 'capturable': False, 'differentiable': False, 'fused': None, 'initial_lr': 0.0001, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 1

In [19]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xb, yb, tb = get_batch(split)
            logits, loss = model(xb, yb)

            B, T, = yb.shape
            C = n_embed

            logits = logits.view(B*T, -1)
            tb = tb.view(tb.shape[0]*tb.shape[1])
            loss = F.cross_entropy(logits, tb)
            
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [27]:
model.train()
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        lr_ = optimizer.param_groups[0]["lr"]
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, lr {lr_}")

    # sample a batch of data
    xb, yb, tb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)

    B, T, = yb.shape
    C = n_embed

    logits = logits.view(B*T, -1)
    tb = tb.view(tb.shape[0]*tb.shape[1])
    loss = F.cross_entropy(logits, tb)


    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 1.8424, val loss 1.8164, lr 0.0001
step 500: train loss 1.7881, val loss 1.8114, lr 0.0001
step 1000: train loss 1.8002, val loss 1.8107, lr 0.0001
step 1500: train loss 1.8035, val loss 1.8099, lr 0.0001
step 2000: train loss 1.8062, val loss 1.8037, lr 0.0001
step 2500: train loss 1.8156, val loss 1.7988, lr 0.0001
step 3000: train loss 1.8302, val loss 1.8207, lr 0.0001
step 3500: train loss 1.8014, val loss 1.8098, lr 0.0001
step 4000: train loss 1.8094, val loss 1.7967, lr 0.0001
step 4500: train loss 1.8355, val loss 1.8183, lr 0.0001
step 5000: train loss 1.8259, val loss 1.8007, lr 0.0001
step 5500: train loss 1.8357, val loss 1.8045, lr 0.0001
step 6000: train loss 1.8121, val loss 1.8170, lr 0.0001
step 6500: train loss 1.8150, val loss 1.8269, lr 0.0001
step 7000: train loss 1.8102, val loss 1.8441, lr 0.0001
step 7500: train loss 1.8240, val loss 1.8152, lr 0.0001
step 8000: train loss 1.8072, val loss 1.7784, lr 0.0001
step 8500: train loss 1.8140, val l

In [21]:
# filepath = os.getcwd()+"\\machine_model\\model.pt"
# torch.save(model.state_dict(), filepath)
# print("model saved at:", filepath)

model saved at: d:\Documents\Github\robots\machine_model\model.pt


rough work

In [26]:
filepath = os.getcwd()+"\\machine_model\\model.pt"
model = Transformer()
model.load_state_dict(torch.load(filepath))
model.eval()
m = model.to(device)

In [115]:
english_sentences[0]

tensor([100277,  11245,  24532,    374,    264,   5644,    369,   1005,  25982,
           902,    374,  17551,    439,    264,   1488,   1169,    555,   2231,
          1919,  22145,    477,  14654,    304,    279,  51370,  13116,    320,
         24359,    883,    315,    279,   9699,   6892,    354,  51370,    662,
           220, 100277, 100277, 100277, 100277, 100277, 100277, 100277, 100277,
        100277, 100277, 100277, 100277, 100277, 100277, 100277, 100277, 100277,
        100277, 100277, 100277, 100277, 100277, 100277, 100277, 100277, 100277,
        100277])

In [116]:
sentence = 'iron cement is a ready for use paste which is laid as a fillet by putty knife or finger in the mould edges ( corners ) of the steel ingot mould .'

tok = tokenizer.encode(sentence, allowed_special=specials)
len(tok)
uh_oh = m.translate(sentence)

UnboundLocalError: local variable 'T' referenced before assignment

In [105]:
tokenizer.decode(uh_oh)

'<|START|> Clean and comfortable accommodation with excellent access to the shops and knowledgeableANN city . <|END|>'

In [ ]:
#GLOBALS

block_size = 64 #This is the value of T
batch_size = 16 #This it the value of B
n_embed = 128
dropout = 0.2
n_heads = 8
n_layers = 6

learning_rate = 3e-4

eval_interval = 500
eval_iters = 200
max_iters = 5000

vocab_size = tokenizer.n_vocab
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device is: {device}")

In [ ]:
#BATCH LOADER
n = int(0.9*len(english_sentences))

train_data_en = english_sentences[:n]
val_data_en = english_sentences[n:]

train_data_de = german_sentences[:n]
val_data_de = german_sentences[n:]

def get_batch(split):
    xdata = train_data_en if "train" else train_data_en
    ydata = train_data_de if "train" else val_data_de
    idx = torch.randint(len(xdata), (batch_size,))
    #print(idx)
    x = torch.stack([xdata[i] for i in idx])
    y = torch.stack([ydata[i] for i in idx])

    #shifting our targets by 1 to the right
    t = y[:, 1:]
    #to pad the last dimension of the input tensor, pad has the form (padding_left, padding_right)
    t = F.pad(input = t, pad = (0,1,0,0), mode = 'constant', value = 0)

    #x, y, t = x.to(device), y.to(device), t.to(device)

    return x, y, t


In [ ]:
xb, yb, t = get_batch('train')

xb.shape

In [ ]:
tok_embedding_matrix = nn.Embedding(vocab_size, n_embed)

x = tok_embedding_matrix(xb)
y = tok_embedding_matrix(yb)

In [ ]:
e = Head(head_size = 64, decoder = False)
e = MultiHeadSelfAttention(64, decoder = False)
e = EncoderBlock()

In [ ]:
out_e = e(x)

In [ ]:
d = Head(head_size = 64, decoder = True)
#d = MultiHeadSelfAttention(head_size = 64, decoder = True)
#d = DecoderSelfBlock()

In [ ]:
c = crossHead(head_size = 64)
c = MultiHeadCrossAttention(head_size = 64)
c = DecoderCrossBlock()
d = DecoderCrossBlock()

In [ ]:
a = torch.cat((out_e, out_d), dim = 0)

In [ ]:
a.shape

In [ ]:
m = Transformer()

In [ ]:
m(xb, yb, t)

In [ ]:
encoder = tiktoken.get_encoding("cl100k_base")


In [ ]:
tokenizer.encode("<|PAD|>", allowed_special = specials)

In [ ]:
tokenizer.decode(english_sentences[0].tolist())

In [ ]:
encoding.decode([83, 1609, 5963, 374, 2294, 0])

In [ ]:
# block_size = 10
# num_examples = 5

# en_max = 0 
# with open(os.getcwd()+'\\data\\train_en.txt', 'r', encoding='utf8') as f:
#     idx_en = []
#     sentences_en = []
#     for i in tqdm(range(num_examples)):
#         line = f.readline()
#         line = line.replace("\n", "")
#         len_pad = 0
#         sentence = "<|PAD|> " + (line) + " <|PAD|>"
#         print(sentence)
#         print(tokenizer.encode(sentence, allowed_special = specials))
#         tok_sentence = tokenizer.encode(sentence, allowed_special = specials)
#         print(len(tok_sentence))
#         if len(tok_sentence) > en_max:
#             en_max = len(tok_sentence)
#             print(en_max)

#         if len(tok_sentence) <= block_size:
#             len_pad = block_size - len(tok_sentence)
#             tok_sentence = tok_sentence + len_pad*[100277]
#             assert len(tok_sentence) == block_size, print(len(tok_sentence))
#             #idx_en.append(i)
#             sentences_en.append(tok_sentence)
#         else:
#             sentences_en.append(block_size*[100280])

# print(en_max)    
# print(f"Length of sentences: {len(sentences_en)}")


# de_max = 0 
# with open(os.getcwd()+'\\data\\train_de.txt', 'r', encoding='utf8') as f:
#     idx_de = []
#     sentences_de = []
#     for i in tqdm(range(num_examples)):
#         line = f.readline()
#         line = line.replace("\n", "")
#         len_pad = 0
#         sentence = "<|START|> " + (line) + " <|END|>"
#         tok_sentence = tokenizer.encode(sentence, allowed_special = specials)
#         if len(tok_sentence) > de_max:
#             de_max = len(tok_sentence)
#             print(de_max)

#         if len(tok_sentence) <= block_size:
#             len_pad = block_size - len(tok_sentence)
#             tok_sentence = tok_sentence + len_pad*[100277]
#             assert len(tok_sentence) == block_size, print(len(tok_sentence))
#             #idx_en.append(i)
#             sentences_de.append(tok_sentence)
#         else:
#             sentences_de.append(block_size*[100280])
            
# print(de_max)  
# print(f"Length of sentences: {len(sentences_de)}")

In [ ]:
# sentences = np.array([sentences_en, sentences_de])

# idx = np.where(sentences == 100280)
# print(idx[1])

In [ ]:
# sentences = np.delete(sentences, idx[1], axis = 1)

In [ ]:
# sentences

In [ ]:
def split_zero_indices(lst):
    tensor = torch.tensor(lst)
    zero_indices = torch.nonzero(tensor == 0).squeeze()
    split_indices = torch.nonzero(torch.diff(zero_indices) != 1).squeeze() + 1
    split_lists = torch.split(zero_indices, split_indices)

    return [list(indices.numpy()) for indices in split_lists]

# Example usage
list1 = [[1, 2, 4, 0, 0, 0],[0, 4, 0, 2, 5, 8]]
list2 = [0, 4, 0, 2, 5, 8]


In [ ]:
p = torch.tensor(list1)
zero_indices = torch.nonzero(p == 0).squeeze()

In [ ]:
zero_indices

In [ ]:
asdf = [3,4,5]

In [ ]:
mask = torch.ones(2, 6, 5)

In [ ]:
mask

In [ ]:
zero_indices[:,0]

In [ ]:
zero_indices[:,1]

In [ ]:
mask[zero_indices[:,0],zero_indices[:,1], :] = 0

In [ ]:
mask

padding testing

In [ ]:
#BATCH LOADER
n = int(0.9*len(english_sentences))

train_data_en = english_sentences[:n]
val_data_en = english_sentences[n:]

train_data_de = german_sentences[:n]
val_data_de = german_sentences[n:]

def get_batch(split):
    xdata = train_data_en if "train" else train_data_en
    ydata = train_data_de if "train" else val_data_de
    idx = torch.randint(len(xdata), (2,))
    #print(idx)
    x = torch.stack([xdata[i] for i in idx])
    y = torch.stack([ydata[i] for i in idx])

    #shifting our targets by 1 to the right
    t = y[:, 1:]
    #to pad the last dimension of the input tensor, pad has the form (padding_left, padding_right)
    t = F.pad(input = t, pad = (0,1,0,0), mode = 'constant', value = 0)

    #x, y, t = x.to(device), y.to(device), t.to(device)

    return x, y, t


In [ ]:
xb, yb, t = get_batch('train')

xb.shape

In [ ]:
tok_embedding_matrix = nn.Embedding(vocab_size, 3)

x = tok_embedding_matrix(xb)
y = tok_embedding_matrix(yb)

In [ ]:
xb

In [ ]:
zero_indices = torch.nonzero(xb == 0).squeeze()

In [ ]:
zero_indices

In [ ]:
pad = torch.ones(x.shape)

In [ ]:
pad

In [ ]:
pad[zero_indices[:,0],zero_indices[:,1], :] = 0

In [ ]:
pad

In [ ]:
x

In [ ]:
x_padded = x * pad

In [ ]:
x_padded